# Install Environment

In [ ]:
# main package
! pip3 install playwright openai

# playwright install web driver
! playwright install

# Prompt Template

In [ ]:
def write_file(filename, text):
	try:
		with open(filename,"w", encoding="utf-8") as f:
			f.write(text)
	except Exception as e:
		print(e)
		print("write failed.")

def read_file(filename):
	try:
		with open(filename,"r", encoding="utf-8") as f:
			content = f.read()
		return content
	except Exception as e:
		print(e)
		print("read failed.")

In [ ]:
prompt = read_file("prompt_template.txt")
print(prompt)

# Python Package

In [ ]:
#!/usr/bin/env python3
# from playwright.sync_api import sync_playwright
#from playwright.async_api import async_playwright  # jupter need to use async 
from playwright.sync_api import playwright, sync_playwright
import time
from sys import argv, exit, platform
import openai
import os

# Playwright Testing

In [3]:
from playwright.async_api import async_playwright # playwright use async and await method

# __init__
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()
await page.set_viewport_size({"width": 1280, "height": 1080})

# go_to_page
await page.goto("http://google.com")
client = await page.context.new_cdp_session(page)
page_element_buffer = {}

# Get the current page content
tree = await client.send(
          "DOMSnapshot.captureSnapshot",
          {"computedStyles": [], "includeDOMRects": True, "includePaintOrder": True},
    )


In [ ]:
tree

In [ ]:
def add_to_hash_tree(node_names, parent, hash_tree, tag, node_id, node_name, parent_id):
			parent_id_str = str(parent_id)
			if not parent_id_str in hash_tree:
				parent_name = strings[node_names[parent_id]].lower()
				grand_parent_id = parent[parent_id]

				add_to_hash_tree(
					hash_tree, tag, parent_id, parent_name, grand_parent_id
				)

			is_parent_desc_anchor, anchor_id = hash_tree[parent_id_str]

			# even if the anchor is nested in another anchor, we set the "root" for all descendants to be ::Self
			if node_name == tag:
				value = (True, node_id)
			elif (
				is_parent_desc_anchor
			):  # reuse the parent's anchor_id (which could be much higher in the tree)
				value = (True, anchor_id)
			else:
				value = (
					False,
					None,
				)  # not a descendant of an anchor, most likely it will become text, an interactive element or discarded

			hash_tree[str(node_id)] = value

			return value

In [16]:
strings	 	= tree["strings"] # 整個 html 裏面 element 對應的 string_name
document 	= tree["documents"][0]
nodes 		= document["nodes"]


In [34]:
# print_node_detail
node_names 	= nodes["nodeName"]
node_types 	= nodes["nodeType"]
node_attributes = nodes["attributes"]
node_parents = nodes["parentIndex"]
black_listed_elements = set(["html", "head", "title", "meta", "iframe", "body", "script", "style", "path", "svg", "br", "::marker",])
for nodeIndex, stringIndex in enumerate(node_names):
    
    if strings[stringIndex].lower() in black_listed_elements: 
        continue
    print(f"node_index:{nodeIndex}, node_string:{stringIndex}")
    print(f"node_name:{strings[stringIndex]}, node_parent:{node_parents[nodeIndex]}, node_types:{node_types[nodeIndex]}, node_attributs:{node_attributes[nodeIndex]}")
    for [attribute_index, attribute_value_index] in zip(*(iter(node_attributes[nodeIndex]),) * 2):
        print(attribute_index, attribute_value_index)
        print(strings[attribute_index], strings[attribute_value_index])
    print('\n')

node_index:0, node_string:4
node_name:#document, node_parent:-1, node_types:9, node_attributs:[]


node_index:8, node_string:23
node_name:#text, node_parent:7, node_types:3, node_attributs:[]


node_index:11, node_string:23
node_name:#text, node_parent:10, node_types:3, node_attributs:[]


node_index:12, node_string:23
node_name:#text, node_parent:3, node_types:3, node_attributs:[]


node_index:14, node_string:23
node_name:#text, node_parent:13, node_types:3, node_attributs:[]


node_index:15, node_string:23
node_name:#text, node_parent:3, node_types:3, node_attributs:[]


node_index:17, node_string:23
node_name:#text, node_parent:16, node_types:3, node_attributs:[]


node_index:18, node_string:23
node_name:#text, node_parent:3, node_types:3, node_attributs:[]


node_index:20, node_string:23
node_name:#text, node_parent:3, node_types:3, node_attributs:[]


node_index:22, node_string:23
node_name:#text, node_parent:21, node_types:3, node_attributs:[]


node_index:24, node_string:23
node

In [36]:
def get_attributes(node_attribute, attributes):
  result = {}

  for [attribute_index, attribute_value_index] in zip(*(iter(node_attribute),) * 2):
    if attribute_value_index < 0:
      continue
    attribute = strings[attribute_index]
    attribute_value = strings[attribute_value_index]

    if attribute in attributes:
      result[attribute] = attribute_value
      attributes.remove(attribute)

      if attributes == []:
        return result

  return result

In [38]:
def add_to_hash_tree(hash_tree, tag, node_id, node_name, parent_id):
			parent_id_str = str(parent_id)
			if not parent_id_str in hash_tree:
				parent_name = strings[node_names[parent_id]].lower()
				grand_parent_id = node_parents[parent_id]

				add_to_hash_tree(
					hash_tree, tag, parent_id, parent_name, grand_parent_id
				)

			is_parent_desc_anchor, anchor_id = hash_tree[parent_id_str]

			# even if the anchor is nested in another anchor, we set the "root" for all descendants to be ::Self
			if node_name == tag:
				value = (True, node_id)
			elif (
				is_parent_desc_anchor
			):  # reuse the parent's anchor_id (which could be much higher in the tree)
				value = (True, anchor_id)
			else:
				value = (
					False,
					None,
				)  # not a descendant of an anchor, most likely it will become text, an interactive element or discarded

			hash_tree[str(node_id)] = value

			return value

In [47]:
is_clickable = set(nodes["isClickable"]["index"])
is_clickable

[4,
 5,
 6,
 12,
 13,
 13,
 13,
 22,
 23,
 24,
 24,
 23,
 23,
 24,
 23,
 23,
 24,
 23,
 23,
 24,
 23,
 24,
 23,
 37,
 23,
 24,
 44,
 37,
 23,
 52,
 37,
 23,
 59,
 59,
 37,
 23,
 66,
 23,
 66,
 23,
 59,
 59,
 59,
 59,
 59,
 59,
 66,
 23,
 59,
 66,
 23,
 59,
 59,
 59,
 59,
 66,
 114,
 119,
 66,
 124,
 23,
 59,
 59,
 37,
 23,
 59,
 37,
 23,
 134,
 59,
 37,
 23,
 150,
 59,
 37,
 23,
 37,
 23,
 156,
 23,
 59,
 59,
 59,
 37,
 23,
 124,
 124,
 59,
 37,
 23,
 124,
 59,
 124,
 124,
 59,
 37,
 23,
 124,
 124,
 59,
 59,
 37,
 23,
 59,
 186,
 23,
 59,
 23,
 59,
 37,
 23,
 37,
 23,
 59,
 59,
 59,
 37,
 23,
 59,
 124,
 114,
 119,
 59,
 37,
 23,
 59,
 234,
 59,
 37,
 23,
 23,
 37,
 23,
 23,
 37,
 23,
 23,
 59,
 23,
 59,
 23,
 124,
 114,
 119,
 23,
 23,
 124,
 23,
 23,
 37,
 23,
 59,
 114,
 119,
 119,
 119,
 119,
 37,
 23,
 59,
 114,
 327,
 330,
 331,
 331,
 119,
 119,
 119,
 59,
 37,
 23,
 59,
 59,
 59,
 59,
 59,
 59,
 37,
 23,
 59,
 361,
 37,
 23,
 59,
 37,
 23,
 369,
 59,
 59,
 59,
 59,
 134,
 59,


In [ ]:
node_names 	= nodes["nodeName"]
node_values = nodes["nodeValue"]
node_types 	= nodes["nodeType"] # 用不到, 比較難拿來分辨 node 的 element
node_attributes = nodes["attributes"]
node_parents = nodes["parentIndex"]

black_listed_elements = set(["html", "head", "title", "meta", "iframe", "body", "script", "style", "path", "svg", "br", "::marker",])

anchor_ancestry = {"-1": (False, None)}
button_ancestry = {"-1": (False, None)}
is_clickables = nodes["isClickable"]["index"]
for nodeIndex, stringIndex in enumerate(node_names):
    node_name = strings[stringIndex]
    node_value = strings[node_values[nodeIndex]]
    #print(nodeIndex, node_name)
    if node_name.lower() in black_listed_elements: 
        continue
    print(node_values[nodeIndex])
    print(f"nodeIndex:{nodeIndex}, node_name:{node_name}, node_value:{node_value}")
    node_attribute = node_attributes[nodeIndex]
    element_attributes = get_attributes(node_attribute, ["type", "placeholder", "aria-label", "title", "alt"])
    print(element_attributes)
    is_clickable = nodeIndex in is_clickables
    print(f"is_clickable:{is_clickable}")
    print("\n")
    # for [attribute_index, attribute_value_index] in zip(*(iter(node_attribute),) * 2):
    #     print(attribute_index, attribute_value_index)
    #     if attribute_value_index < 0:
    #         continue
    #     print(strings[attribute_index], strings[attribute_value_index])
    # print('\n')
    